## pip

In [15]:
!pip install selenium

In [2]:
!pip install pymystem3

In [3]:
!pip install wordcloud

In [4]:
!pip install scikit-learn

## Импортируем библиотеки

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from pymystem3 import Mystem
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
import time
import requests
import pandas as pd
import nltk
import re
import matplotlib.pyplot as plt

## Парсер

In [6]:
# Опции для Chrome
options = webdriver.ChromeOptions()
options.add_argument("--ignore-certificate-errors")
options.add_argument("--allow-insecure-localhost")
driver = webdriver.Chrome(options=options)

In [7]:
#Основа парсера
try:
    driver.get("https://www.riatomsk.ru/novosti")# Переход на сайт

    # Прокликивание кнопки "Далее" n-раз
    for _ in range(450):
        try:
            button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "nextLink"))
            )
            button.click()
            time.sleep(9)  # время N ожидания
        except Exception as e:
            print(f"Ошибка при нажатии кнопки: {e}")
            break

    # Получение ссылок на новости
    time.sleep(9)  # N пауза для полной загрузки страницы
    soup = BeautifulSoup(driver.page_source, "html.parser")
    news_links = []

    # Поиск всех новостей на странице
    articles = soup.find_all("a", class_="rubNewItem")

    if articles:
        for article in articles:
            link = article.get("href")  # Получаем атрибут 'href' напрямую
            if "2021" in link: # Фильтрация по N году
                news_links.append(link)
                print(f"Собрана ссылка: {link}")
    else:
        print("Не удалось найти статьи на странице.")

except Exception as e:
    print(f"Произошла ошибка: {e}")

finally:
    driver.quit()  # Закрытие браузера

Ошибка при нажатии кнопки: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=131.0.6778.85); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
#0 0x5b86f3ba234a <unknown>
#1 0x5b86f36b86e0 <unknown>
#2 0x5b86f36be4a3 <unknown>
#3 0x5b86f36c0968 <unknown>
#4 0x5b86f36c09f3 <unknown>
#5 0x5b86f370941e <unknown>
#6 0x5b86f37086df <unknown>
#7 0x5b86f36fc29a <unknown>
#8 0x5b86f372b462 <unknown>
#9 0x5b86f36fba18 <unknown>
#10 0x5b86f372b62e <unknown>
#11 0x5b86f3749ed7 <unknown>
#12 0x5b86f372b203 <unknown>
#13 0x5b86f36f9cc0 <unknown>
#14 0x5b86f36fac9e <unknown>
#15 0x5b86f3b6fd3b <unknown>
#16 0x5b86f3b73cc2 <unknown>
#17 0x5b86f3b5cb6c <unknown>
#18 0x5b86f3b74837 <unknown>
#19 0x5b86f3b4210f <unknown>
#20 0x5b86f3b915a8 <unknown>
#21 0x5b86f3b91770 <unknown>
#22 0x5b86f3ba11c6 <unknown>
#23 0x7a675089ca94 <unknown>
#24

In [8]:
# запись ссылок в файл
if news_links:
    try:
        with open("news_links_2021.txt", "w", encoding="utf-8") as file:
            for link in news_links:
                file.write(link + "\n")
        print("Ссылки на новости за 2021 год успешно записаны.")
    except Exception as e:
        print(f"Ошибка при записи в файл: {e}")
else:
    print("Ссылки не были найдены для записи.")

Ссылки на новости за 2021 год успешно записаны.


In [9]:
# чтение ссылок из файла
with open("news_links_2021.txt", "r", encoding="utf-8") as file:
    links = file.readlines()

In [10]:
type(links)

list

# Парсинг данных из статей

In [11]:
# список данных
info = []  

for link in links:
    link = link.strip()
    try:
        response = requests.get(link)
        if response.status_code == 200:
            soup1 = BeautifulSoup(response.text, "html.parser")

            # заголовок
            title_h1 = soup1.find("div", class_="statInfoName")
            title = title_h1.text.strip() if title_h1 else None

            # контент
            content_div = soup1.find("div", class_="statAbout")
            text = content_div.get_text(separator=" ").strip() if content_div else None

            # дата и время
            date_div = soup1.find("div", style="font-size: 16px; margin-top: 10px;")
            if date_div:
                date_text = date_div.get_text(separator=" ").strip()
                time, date = date_text[:5], date_text[5:40]
            else:
                time, date = None, None

            # добавить в список
            info.append({
                "url": link,
                "title": title,
                "text": text,
                "time": time,
                "date": date
            })
            print(link, title, text, time, date, sep="\n")

    except Exception as e:
        print(f"Ошибка при обработке ссылки {link}: {e}")  

Ошибка при обработке ссылки /article/20241203/brucellez-viyavlen-u-tomskih-ovec-vpervie-s-2021g-vveden-karantin/: Invalid URL '/article/20241203/brucellez-viyavlen-u-tomskih-ovec-vpervie-s-2021g-vveden-karantin/': No scheme supplied. Perhaps you meant https:///article/20241203/brucellez-viyavlen-u-tomskih-ovec-vpervie-s-2021g-vveden-karantin/?


# проверка содержимого

In [12]:
info

[]

In [13]:
df = pd.DataFrame(info)

In [14]:
print(str(df['text'][1]))

KeyError: 'text'

In [ ]:
df.to_csv(r'RIA_articles_2021.csv', index= False )

## Preprocessing

In [ ]:
# данные
df = pd.read_csv(r"RIA_articles_2021.csv", encoding='UTF-8')
df['prep'] = df['text']  

In [ ]:
# загрузка стоп-слов
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('russian'))

In [ ]:
# удаление Стоп-слов
def remove_stopwords(text):
    # Удаление Ничего  
    if not isinstance(text, str):
        return ""
    # Удаление стоп-слов
    return " ".join([word for word in text.split() if word not in STOPWORDS])

In [ ]:
# обработка
df['prep'] = (
    df['prep']
    .str.lower()  # Регистр ниж.
    .apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))  # пунктуация
    .apply(remove_stopwords)  # Удаление стоп-слов
)

In [ ]:
# сохранение Preprocessing
df.to_csv("RIA_articles_prep_2021.csv", index=False) # если меняете год в парсере, не забудьте менять годы в CSV файлах!
print("успешно сохранён.")

## Mystem

In [ ]:
# данные
df = pd.read_csv("RIA_articles_prep_2021.csv", encoding='UTF-8') # если меняете год в парсере, не забудьте менять годы в CSV файлах!

In [ ]:
# запуск Mystem
m = Mystem()

Заметки.
Если у вас на этом тапе ошибка, то скачайте mystem и перенесите его в /local/bin. 
Команды: 
mkdir -p ~/.local/bin
mv mystem ~/.local/bin/
chmod +x ~/.local/bin/mystem

In [ ]:
print(df.columns)

In [ ]:
def lemmatize_text(text):
    if pd.isna(text):  # Проверка на NaN
        return ""
    return ''.join(m.lemmatize(text))

# лемматизация
df['lemmas'] = df['prep'].apply(lemmatize_text)

# стоп-слова из библиотеки
nltk.download('stopwords')
STOPWORDS = set(stopwords.words('russian'))

In [ ]:
# список стоп слов от Федора
word_list = {"томск", "томский", "томич", "томичи", "томичка", "голубев", "масляева", "соколов", "мазуров", "мазур",
             "прошкина", "планкина", "матвиевский", "риа", "агентство", "миллион", "год", "сообщаться", "ранее",
             "сообщалось", "сообщать", "скр", "прокуратура", "администрация", "мэрия", "мэрии", "департамент",
             "пресс-службы", "область", "журналист", "корреспондент", "рубль", "также", "отмечается", "день", "весь",
             "регион", "тысяча", "свой", "россия", "бюджет", "вопрос", "рассказывать", "время", "комитет",
             "проводить", "добавлять", "около", "решение", "редакция", "город", "улица", "месяц", "погода", "тепло", 
             "который", "это", "минус"}

STOPWORDS_1 = STOPWORDS.union(word_list)

In [ ]:
# удаление стоп-слов от федора
def remove_stopwords(text):
    if pd.isna(text):  # Проверка на NaN
        return ""
    return " ".join([word for word in text.split() if word not in STOPWORDS_1])

df['stop'] = df['lemmas'].apply(remove_stopwords)

In [ ]:
# удаление цифр и пунктуации
def clean_text(text):
    if pd.isna(text):  # Проверка на NaN
        return ""
    return re.sub(r'[^\w\s]+|[\d]+', '', text)

df['stop_num'] = df['stop'].apply(clean_text)

In [ ]:
# удаление символа "r" в тексте
df['stop_num'] = df['stop_num'].str.replace('r', '')

In [ ]:
# сохранение Mystem
df.to_csv("RIA_articles_lemmas_2021.csv", index=False) # если меняете год в парсере, не забудьте менять годы в CSV файлах!
print("успешно сохранён.")

## TF-IDF

In [ ]:
# вывод графика
%matplotlib inline

In [ ]:
# данные
df = pd.read_csv(r"RIA_articles_lemmas_2021.csv", encoding='UTF-8') # если меняете год в парсере, не забудьте менять годы в CSV файлах!

In [ ]:
# Токенизация
tokenizer = RegexpTokenizer(r'\w+')
df['text_prep'] = df['stop_num'].apply(lambda x: tokenizer.tokenize(str(x)))

In [ ]:
# единый список
all_texts = [word for words_list in df['text_prep'] for word in words_list]

# удаление символов 'r' и стоп-слов (Еще раз, на всякий)
add_stopwords = {'nan', 'r'}
all_texts_cleaned = [word.replace('r', '').strip() for word in all_texts if word not in add_stopwords]

# частота слов
fdist = FreqDist(all_texts_cleaned)
print(fdist.most_common(5))  # Топ-5 самых частотных слов
fdist.plot(30, cumulative=False)  # График частотности

In [ ]:
# облако
text_raw = " ".join(all_texts_cleaned)
wordcloud = WordCloud().generate(text_raw)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

In [ ]:
# вывод облака
plt.show()

In [ ]:
# текст для TF-IDF
df['text_prep_str'] = df['text_prep'].apply(lambda x: " ".join(x))

# вычисление TF-IDF
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['text_prep_str'])

In [ ]:
# слова и их TF-IDF
feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_values = tfidf_matrix.sum(axis=0).A1  # Суммируем значения TF-IDF по всем документам

In [ ]:
# создание нового csv
words_tfidf_df = pd.DataFrame({
    "Слово": feature_names,
    "TF-IDF": tfidf_values
}).sort_values(by="TF-IDF", ascending=False)

In [ ]:
# сохранение TF-IDF
words_tfidf_df.to_csv("RIA_articles_tfidf_2021.csv", index=False) # если меняете год в парсере, не забудьте менять годы в CSV файлах!
print("Файл 'RIA_articles_tfidf.csv' успешно сохранён.")